# Get Tweets for n no. of days

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas

from datetime import date, timedelta
import datetime


import sys
old_stdout = sys.stdout

log_file = open("message.log","w")

sys.stdout = log_file

tweets_list2 = []
num_of_tweets = 5000
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2020, 1, 1)
end_date = date(2020, 1, 4)

for single_date in daterange(start_date, end_date):
    
    since_ = single_date.strftime("%Y-%m-%d")
    next_day = single_date + datetime.timedelta(days=1)
    until_ = next_day.strftime("%Y-%m-%d")

    print(single_date.strftime("%Y-%m-%d"),' to ', next_day.strftime("%Y-%m-%d"))

    j = 0

    # since_ = '2020-01-01'
    # until_ = '2020-01-02'

    Apple_related_list = ['$Aapl', '$aapl', '@Apple', '#AAPL','@Aapl', '@apple', '#aapl']
    # Apple_related_list = ['google', 'trump']
    parsed_terms = []
    for term in Apple_related_list:
        parsed_terms.append(term)
        if j <= num_of_tweets:
            print('Working with {}'.format(term))
            
            for i,tweet in enumerate(sntwitter.TwitterSearchScraper('{} since:{} until:{}'.format(term, since_, until_)).get_items()):
                if j>num_of_tweets:
                    print('Got my {} tweets for date: {} from the term/terms: {}'.format(num_of_tweets, since_, parsed_terms))
                    break
                #if(tweet.user.verified == True and tweet.lang == 'en'):
                if(tweet.lang == 'en'):
                    j+=1
                    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.retweetCount, tweet.lang, tweet.user.verified])
                
                
    print('Final val of j: {}'.format(j))    
    if(j < num_of_tweets):
        print('Got {} tweets for date: {} from the term/terms: {}. Insufficient no. of valid Tweets!'.format(j, since_, parsed_terms))
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Retweet_count', 'Language', 'Verified'])
filename = 'SNS_scraped_tweets.csv'
tweets_df2.to_csv(filename)

sys.stdout = old_stdout

log_file.close()

# Push CSV data to Postgresql

In [73]:
import tweepy
import psycopg2
import pandas as pd
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.bool_, psycopg2._psycopg.AsIs)

# Initial DB table setup
# Connecting to DB
conn = psycopg2.connect(host = "localhost", database = "my_first_local_db", port = "5432", user = "postgres", password = "Local")

# Table creation only if it doesen't exist already!
# Tweet_Id should be unique, this prevent duplicate entries into the DB
commands = (# Table 1         
            '''Create Table IF NOT EXISTS Tweets_table(
                id SERIAL PRIMARY KEY,
                date DATE,
                tweet_Id BIGINT UNIQUE, 
                content TEXT,
                user_Name TEXT,
                retweet_Count BIGINT,  
                language TEXT,
                verified TEXT );''')

# Creating a cursor to execute sql commands
cur = conn.cursor()
cur.execute(commands)

conn.commit()
cur.close()
conn.close()


# Function to push rows to DB
def dbconnect(date, tweet_id, content, user_name, retweet_count, language, verified):

    conn = psycopg2.connect(host = "localhost", database = "my_first_local_db", port = "5432", user = "postgres", password = "Local")
    cur = conn.cursor()

    command = '''INSERT INTO Tweets_table (date, tweet_id, content, user_name, retweet_count, language, verified) VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;'''
    cur.execute(command, (date, tweet_id, content, user_name, retweet_count, language, verified))

    conn.commit()
    cur.close()
    conn.close()



# reading the csv into pnadas df
tweet_df = pd.read_csv("SNS_scraped_tweets.csv")

# Datetime	Tweet Id	Text	Username	Retweet_count	Language	Verified
for idx in tweet_df.index:
    # read each row of csv and push it to DB
    dbconnect(tweet_df['Datetime'][idx], tweet_df['Tweet Id'][idx], tweet_df['Text'][idx], tweet_df['Username'][idx], tweet_df['Retweet_count'][idx], tweet_df['Language'][idx], tweet_df['Verified'][idx] )
    
                
             